# Naive Bayes Implementation

Referencias:
- http://artificiencia.com/aprenda/algoritmo-naive-bayes/
- https://chrisalbon.com/machine_learning/naive_bayes/naive_bayes_classifier_from_scratch/ (DONE)
- https://hackinganalytics.com/2016/10/13/desvendando-o-classificador-naive-bayes/ (DONE)
- https://towardsdatascience.com/naive-bayes-in-machine-learning-f49cc8f831b4


1. Dado um dataset, calcular probabilidade de cada classe
    - P(Jogar = Sim) = 9/14;
    - p(Jogar = Nao) = 5/14;
2. Calcular probabilidade de cada atributo
    - P(Genero = Sol | Target = Sim) = 2/9;
    - Continua...
    
    
## Dúvidas
R: Sobre o treinamento, o que é o treinamento do Navie Bayes.<br>
R: Treinamento é as `probabilidade condicional`

In [1]:
import pandas as pd
import numpy as np

dataset = pd.read_csv('dataset/dataset.csv')

dataset.head(1)

,Genero,Idade,Escolaridade,Profissao,Target
0,F,a - Ate 25 anos,Fundamental,b,0


### Funções auxiliares
- P(A | B):
    - Função auxiliar para fazer `probabilidade condicional`
- P(A):
    - Função auxiliar para fazer `probabilidade condicional`

In [2]:
def p_given_q(ds, p, q, val_p, val_q):
    p_f_zero = ds[(dataset[p]==val_p) & (ds[q]==val_q)][p].count()/ds[p][ds[q]==val_q].count()
    return p_f_zero

def p_q(ds, p,val_p):
    return ds[p][ds[p]==val_p].count()/ds[p].count()

#p_given_q('Weather','Target','sunny',1)

### Probabilidade de Target ser 0 ou 1

In [3]:
# Counters
n_tg_zero = dataset['Target'][dataset['Target']==0].count()
n_tg_one = dataset['Target'][dataset['Target']==1].count()

n_all = dataset['Target'].count()

In [4]:
p_tg_zero = n_tg_zero/n_all
p_tg_one = n_tg_one/n_all

### Listas com possibilidades

In [5]:
#Training

list_columns = list(dataset)
list_columns.pop()
list_value_tg = list(dataset['Target'].unique())
list_results = list()

#for column in list_columns:
#   for value_col in list(dataset[column].unique()):
#       for value_tg in list_value_tg:
#           list_results.append((column+'='+str(value_col)+' dado Target='+ str(value_tg), p_given_q(column,'Target',value_col,value_tg)))
# list_results

### Formula para predições
MAP(h) = max (P (o|h) x P(h))

In [6]:
# Calcule predicts
ds_train=dataset.sample(frac=0.8,random_state=200)
ds_test=dataset.drop(ds_train.index)

count_correct = 0
count_incorrect = 0

for index, row in ds_test.iterrows():
    classes = list()
    for value_tg in list_value_tg:
        prob = p_q(ds_train,'Target',value_tg)
        for column in list_columns:
            prob *= p_given_q(ds_train,column,'Target',row[column],value_tg)
        classes.append((value_tg, prob))
        
    #Result
    result = max(classes,key=lambda p: p[1])
    if result[0] == row['Target']:
        count_correct += 1
    else:
        count_incorrect += 1

print(count_correct)
print(count_incorrect)
        

/usr/lib/python3/dist-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


151
49


In [7]:
print(n_tg_one)
print(n_tg_zero)

772
228
